<a href="https://colab.research.google.com/github/ashishmathew0297/movie_rating_prediction_cis550_final_project/blob/main/Nikita/Pi_data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from math import floor, ceil
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
from ast import literal_eval

In [ ]:
pd.set_option("display.max_columns", None)


In [ ]:
imdb_dataset = pd.read_csv('/content/drive/MyDrive/drive-download-20231202T144505Z-001/imdb_movie_dataset.csv')
imdb_dataset.head()

,Unnamed: 0,tconst,primaryTitle,isAdult,releaseYear,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N,Short,region,actor,actress,cinematographer,composer,director,editor,producer,production_designer,self,writer,averageRating,numVotes
0,0,tt0000502,Bohemios,False,1905,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,"\N,ES","nm0215752,nm0252720",NaN,NaN,NaN,nm0063413,NaN,NaN,NaN,NaN,"nm0675388,nm0063413,nm0657268",4.1,15
1,1,tt0000574,The Story of the Kelly Gang,False,1906,70,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"HU,AU,RS,GB,\N,SG,US,DE,AU","nm0846894,nm1431224,nm3002376",nm0846887,nm0675239,nm2421834,nm0846879,NaN,"nm0317210,nm0425854,nm0846911",NaN,NaN,nm0846879,6.0,855
2,2,tt0000591,The Prodigal Son,False,1907,90,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"FR,\N,US","nm0906197,nm0332182","nm1323543,nm1759558",NaN,NaN,nm0141150,NaN,NaN,NaN,NaN,nm0141150,5.0,21
3,3,tt0000615,Robbery Under Arms,False,1907,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"\N,AU","nm3071427,nm0581353,nm0888988,nm0240418,nm0346387",nm0218953,nm0167619,NaN,nm0533958,NaN,NaN,NaN,NaN,"nm0533958,nm0092809",4.3,25
4,4,tt0000630,Hamlet,False,1908,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"IT,\N,FI,US,FI",NaN,nm0624446,NaN,NaN,nm0143333,NaN,nm0209738,NaN,NaN,nm0000636,2.9,27


In [ ]:
personalInfo_dataset = pd.read_csv('/content/drive/MyDrive/personnel_information.csv')

personalInfo_dataset['knownForTitles'] = personalInfo_dataset['knownForTitles'].convert_dtypes(convert_string=True)

personalInfo_dataset.info()

# personalInfo_dataset.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127583 entries, 0 to 1127582
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Unnamed: 0         1127583 non-null  int64  
 1   level_0            1127583 non-null  int64  
 2   index              1127583 non-null  int64  
 3   nconst             1127583 non-null  object 
 4   primaryName        1127583 non-null  object 
 5   birthYear          1127583 non-null  object 
 6   deathYear          1127583 non-null  object 
 7   primaryProfession  1033360 non-null  object 
 8   knownForTitles     1127583 non-null  string 
 9   pi_mean            727578 non-null   float64
 10  pi_median          727578 non-null   float64
 11  pi_std             317054 non-null   float64
dtypes: float64(3), int64(3), object(5), string(1)
memory usage: 103.2+ MB


In [ ]:
personalInfo_dataset.dropna(subset=['primaryProfession','pi_mean','pi_median','pi_std'], inplace=True)
cols = [0,1,2]
personalInfo_dataset.drop(personalInfo_dataset.columns[cols], axis=1, inplace=True)
personalInfo_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313163 entries, 28 to 1127582
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   nconst             313163 non-null  object 
 1   primaryName        313163 non-null  object 
 2   birthYear          313163 non-null  object 
 3   deathYear          313163 non-null  object 
 4   primaryProfession  313163 non-null  object 
 5   knownForTitles     313163 non-null  string 
 6   pi_mean            313163 non-null  float64
 7   pi_median          313163 non-null  float64
 8   pi_std             313163 non-null  float64
dtypes: float64(3), object(5), string(1)
memory usage: 23.9+ MB


In [17]:
personalInfo_dataset['knownForTitles'] = personalInfo_dataset['knownForTitles'].str.split(',')
# personalInfo_dataset.dropna(subset=['knownForTitles'], inplace=True)
personalInfo_dataset= personalInfo_dataset.explode(column='knownForTitles')
personalInfo_dataset = personalInfo_dataset.rename(columns={'knownForTitles': 'tconst'})
personalInfo_dataset.info()
# personalInfo_dataset)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1185837 entries, 28 to 1127582
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   nconst             1185837 non-null  object 
 1   primaryName        1185837 non-null  object 
 2   birthYear          1185837 non-null  object 
 3   deathYear          1185837 non-null  object 
 4   primaryProfession  1185837 non-null  object 
 5   tconst             1185837 non-null  object 
 6   pi_mean            1185837 non-null  float64
 7   pi_median          1185837 non-null  float64
 8   pi_std             1185837 non-null  float64
dtypes: float64(3), object(6)
memory usage: 90.5+ MB


In [18]:
personalInfo_dataset.to_csv('/content/drive/MyDrive/drive-download-20231202T144505Z-001/personalInfoExploded_.csv')